In [1]:
from collections import Counter
import numpy as np
from pymatgen.io.vasp import Poscar
from potentials import BuckinghamPotential
from potential_parameters import BuckinghamParameter
from atom_types import AtomType

def types_from_structure(structure):
    elements = Counter(structure.species)
    for e in elements:
        print(e.name)
        print(type(e.name))


In [2]:
def pot_params():
    parameters = []
    parameters.append(BuckinghamParameter('Li_O_a', 'a', 1.0))
    parameters.append(BuckinghamParameter('Li_O_rho', 'rho', 0.1))
    parameters.append(BuckinghamParameter('Li_O_c', 'c', 0.0))
    return parameters

parameters = pot_params()
parameters

In [3]:
potentials = {'Li-O':[10.00, 0.1, 0.0]}
potentials

{'Li-O': [10.0, 0.1, 0.0]}

In [4]:
def atom_types():
    atom_types = []
    atom_types.append(AtomType(1, 'Li', 'Li', 1.555, 1.0))
    atom_types.append(AtomType(2, 'Ni', 'Ni', 56.111, 3.0))
    atom_types.append(AtomType(3, 'O core', 'O', 14.001, -0.96, 'core'))
    atom_types.append(AtomType(3, 'O shell', 'O', 1.998, -1.04, 'shell'))
    return atom_types

atom_type_list = atom_types()
atom_type_list

In [43]:
label = "O-O spring"
split_label = label.replace('-', ' ').split()
for element in split_label[:2]:
    print(element)

O
O


In [5]:
import mock

In [6]:
at = mock.Mock(AtomType)

In [7]:
at.label = "Li core"

In [8]:
from collections import Counter
import numpy as np
from bonds import Bond
from atoms import Atom
from atom_types import AtomType
from bond_types import BondType

In [9]:
from pymatgen.io.vasp.outputs import Vasprun

vasprun = Vasprun('test_files/test_vasprun.xml')
structure = vasprun.ionic_steps[0]['structure']
structure.add_site_property('forces', np.array(vasprun.ionic_steps[0]['forces']))

/Users/lmm82/.pyenv/versions/3.7.3/lib/python3.7/site-packages/pymatgen/io/vasp/outputs.py:907: UserWarning: No POTCAR file with matching TITEL fields was found in /Users/lmm82/Documents/current_work/lammps_potenial_fitting/test_files
  " was found in {}".format(os.path.abspath(p)))


In [10]:
params = {}
params['core_shell'] = { 'Li': False, 'Ni': False, 'O': True }
params['charges'] = {'Li': +1.0,
                     'Ni': +3.0,
                     'O': {'core':  -2.0,
                           'shell': 0.0}} 
params['masses'] = {'Li': 6.941,
                    'Ni': 58.6934,
                    'O': {'core': 14.3991,
                          'shell': 1.5999} }
params['potentials'] = {'Li-O': [100.00, 0.1, 0.0],
                        'Ni-O': [1000.00, 0.2, 0.0],
                        'O-O':  [10000.00, 0.3, 0.0]}
params['cs_springs'] = {'O-O' : [10.0, 0.0]}

In [11]:
def types_from_structure( structure, core_shell, charges, masses, cs_spring=None, verbose=True ):
    atom_types = []
    bond_types = []
    atom_type_index = 0
    bond_type_index = 0
    elements = Counter(structure.species)
    if verbose:
        print( "Found elements: {}".format( [e.name for e in elements]))
    for e in elements:
        if e.name not in core_shell:
            raise ValueError( '{} not in core_shell dictionary'.format(e.name) )
        if e.name not in charges:
            raise ValueError( '{} not in charges dictionary'.format(e.name) )
        if e.name not in masses:
            raise ValueError( '{} not in masses dictionary'.format(e.name) )
    for e in elements:
        if core_shell[e.name]: # Create two atom_types for core + shell
            atom_type_index += 1
            atom_types.append( AtomType(atom_type_index=atom_type_index,
                                        label='{} core'.format(e.name),
                                        element_type = e.name,
                                        mass=masses[e.name]['core'],
                                        charge=charges[e.name]['core'],
                                        core_shell='core') )
            atom_type_index += 1
            atom_types.append( AtomType(atom_type_index=atom_type_index,
                                        label='{} shell'.format(e.name),
                                        element_type = e.name,
                                        mass=masses[e.name]['shell'],
                                        charge=charges[e.name]['shell'],
                                        core_shell='shell') )
            bond_type_index += 1
            bond_types.append( BondType(bond_type_index=bond_type_index,
                                        label='{}-{} spring'.format(e.name, e.name),
                                        spring_coeff_1=cs_spring['{}-{}'.format(e.name, e.name)][0],
                                        spring_coeff_2=cs_spring['{}-{}'.format(e.name, e.name)][1]))
        else:
            atom_type_index += 1
            atom_types.append( AtomType(atom_type_index=atom_type_index,
                                        label='{}'.format(e.name),
                                        element_type = e.name,
                                        mass=masses[e.name],
                                        charge=charges[e.name] ) )
    return atom_types, bond_types

In [12]:
atom_types, bond_types = types_from_structure( structure, params['core_shell'], params['charges'], params['masses'], params['cs_springs'] )

Found elements: ['Li', 'Ni', 'O']


In [13]:
def atoms_and_bonds_from_structure( structure, atom_types, bond_types ):
    atoms = []
    bonds = []
    atom_types_dict = {}
    atom_types_dict = {at.label: at for at in atom_types}
    bond_types_dict = {}
    bond_types_dict = {bt.label: bt for bt in bond_types}
    atom_index = 0
    bond_index = 0
    molecule_index = 0
    for site in structure:
        molecule_index += 1
        if site.species_string in atom_types_dict: # not core-shell atom
            atom_index += 1
            atom_type = atom_types_dict[site.species_string]
            atoms.append( Atom(atom_index=atom_index,
                               molecule_index=molecule_index,
                               coords=site.coords,
                               atom_forces=site.properties['forces'],
                               atom_type=atom_type) )
        else: # need to handle core + shell
            atom_index += 1
            atom_type = atom_types_dict[site.species_string + ' core']
            atoms.append( Atom(atom_index=atom_index,
                               molecule_index=molecule_index,
                               coords=site.coords,
                               atom_forces=site.properties['forces'],
                               atom_type=atom_type) )
            atom_index += 1
            atom_type = atom_types_dict[site.species_string + ' shell']
            atoms.append( Atom(atom_index=atom_index,
                               molecule_index=molecule_index,
                               coords=site.coords,
                               atom_forces=np.array([0.0, 0.0, 0.0]),
                               atom_type=atom_type) )
            bond_index += 1
            bond_type = bond_types_dict['{}-{} spring'.format( site.species_string, site.species_string )]
            bonds.append( Bond(bond_index=bond_index,
                               atom_indices = [atom_index-1, atom_index],
                               bond_type=bond_type))
    return atoms, bonds 


In [14]:
atoms, bonds = atoms_and_bonds_from_structure( structure, atom_types, bond_types )

In [36]:
len(bonds)

96

In [39]:
bonds[0].bond_type